In [1]:
import pandas as pd
#import dataset 
#önce xml xlsx'e dönüştü
df = pd.read_excel("...your_path...\\sample_dataset.xlsx", 
                   usecols=("Stok Kodu","Fiyat","Satışta","Sayfa Sayısı"))

#Prices are corrected (Previous Sales Strategy was equal to 33% of Turkish Lira Prices)
df["Fiyat"] = df["Fiyat"]/33*100

#If the page number of a book is missing that book is considered 500 aligning with the company owner's decision
df.loc[df["Sayfa Sayısı"] == 0, "Sayfa Sayısı"] = 500


#repricing strategy
#the current currency is selected at 2.5.2023. Price per page, mark-up, and shipping cost are added followingly
df["Fiyat"] = df["Fiyat"]/21.50+ df["Sayfa Sayısı"]*0.01+5+3

#Amazon collects a profit of 15% and 1 euro for each product. 8% VAT is also calculated. The final formula is below
df["standard_price"] = (df["Fiyat"]*100+100)/77

#the column that is no longer needed is dropped
df = df.drop(columns=["Sayfa Sayısı"])

#rename
df.rename({"Stok Kodu": "SKU"}, axis=1)

#max ve min prices
df.loc[df["Satışta"] == "y", "Min.-Price"] = df["Fiyat"]*0.85
df.loc[df["Satışta"] == "y", "Max.-Price"] = df["Fiyat"]*1.2
df.loc[df["Satışta"] == "n", "Min.-Price"] = df["Fiyat"]*3
df.loc[df["Satışta"] == "n", "Max.-Price"] = df["Fiyat"]*4

#the columns that are no longer needed are dropped
df = df.drop(columns=["Fiyat"])
df = df.drop(columns=["Satışta"])

#A strategy was set in Snaptrade in advance considering the rival sellers
df["Strategy"]=1
df["Repricing active"]=1
df.rename({'Stok Kodu': 'SKU'}, axis=1, inplace=True)

#To solve decimal number problems in CSV files, they are removed
df["Min.-Price"] = df["Min.-Price"].fillna(0.0).astype(int)
df["Max.-Price"] = df["Max.-Price"].fillna(0.0).astype(int)
df["SKU"] = df["SKU"].astype(str)

#The excel will be resaved as CSV manually
df.to_excel(r'C:\\Users\\Casper\\Desktop\\snap_prices.xlsx', index=False)